In [1]:
import pandas as pd
import os

os.chdir("./data/stocks_data")

In [2]:
stocks_list = pd.read_csv(r".\stocks_list\SP500HistoricalComponents.csv")
stocks_list = stocks_list.set_index("date")
stocks_list.index = pd.to_datetime(stocks_list.index).to_period("M")
stocks_list["tickers"] = stocks_list["tickers"].str.split(",")
stocks_list = stocks_list.groupby("date").sum()
stocks_list.head()

,tickers
date,
1996-01,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-02,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-03,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-04,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-05,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."


In [3]:
all_stocks = set(stocks_list["tickers"].sum())
all_stocks = sorted(list(all_stocks))
all_stocks[:5]

['A', 'AABA', 'AAL', 'AAMRQ', 'AAP']

In [4]:
result = pd.DataFrame()
for date in stocks_list.index:
    s = pd.Series(index=all_stocks)
    s.loc[stocks_list.loc[date, "tickers"]] = 1
    s.name = date
    result = result._append(s)
result = result.fillna(0).astype(bool).shift(-1).dropna()
result = result.iloc[:-2]
result

,A,AABA,AAL,AAMRQ,AAP,AAPL,ABBV,ABC,ABI,ABKFQ,...,XRX,XTO,XYL,YNR,YRCW,YUM,ZBH,ZBRA,ZION,ZTS
1996-01,False,False,True,True,False,True,False,False,True,False,...,True,False,False,False,True,False,False,False,False,False
1996-02,False,False,True,True,False,True,False,False,True,False,...,True,False,False,False,True,False,False,False,False,False
1996-03,False,False,True,True,False,True,False,False,True,False,...,True,False,False,False,True,False,False,False,False,False
1996-04,False,False,True,True,False,True,False,False,True,False,...,True,False,False,False,True,False,False,False,False,False
1996-05,False,False,True,True,False,True,False,False,True,False,...,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,True,False,True,False,True,True,True,True,False,False,...,False,False,True,False,False,True,True,True,True,True
2023-06,True,False,True,False,False,True,True,True,False,False,...,False,False,True,False,False,True,True,True,True,True
2023-08,True,False,True,False,False,True,True,False,False,False,...,False,False,True,False,False,True,True,True,True,True
2023-09,True,False,True,False,False,True,True,False,False,False,...,False,False,True,False,False,True,True,True,True,True


In [5]:
date_range = pd.date_range(start=pd.to_datetime("1996-01-01"), end=pd.to_datetime("2024-04-01"), freq="M")
result2 = pd.DataFrame(index=date_range.to_period("M"))
result2 = pd.merge(result2, result, left_index=True, right_index=True, how="left")
result2 = result2.infer_objects(copy=False).ffill()
result2

,A,AABA,AAL,AAMRQ,AAP,AAPL,ABBV,ABC,ABI,ABKFQ,...,XRX,XTO,XYL,YNR,YRCW,YUM,ZBH,ZBRA,ZION,ZTS
1996-01,False,False,True,True,False,True,False,False,True,False,...,True,False,False,False,True,False,False,False,False,False
1996-02,False,False,True,True,False,True,False,False,True,False,...,True,False,False,False,True,False,False,False,False,False
1996-03,False,False,True,True,False,True,False,False,True,False,...,True,False,False,False,True,False,False,False,False,False
1996-04,False,False,True,True,False,True,False,False,True,False,...,True,False,False,False,True,False,False,False,False,False
1996-05,False,False,True,True,False,True,False,False,True,False,...,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11,True,False,True,False,False,True,True,False,False,False,...,False,False,True,False,False,True,True,True,True,True
2023-12,True,False,True,False,False,True,True,False,False,False,...,False,False,True,False,False,True,True,True,True,True
2024-01,True,False,True,False,False,True,True,False,False,False,...,False,False,True,False,False,True,True,True,True,True
2024-02,True,False,True,False,False,True,True,False,False,False,...,False,False,True,False,False,True,True,True,True,True


In [7]:
import yfinance as yf

data = yf.download(all_stocks, start="1996-01-01", end="2024-04-30", group_by="column", period="1d")
data.head()

[*********************100%***********************]  1163 of 1163 completed


285 Failed downloads:
['UN', 'HDLM', 'FRX', 'DRE', 'AFS.A', 'ALXN', 'UAWGQ', 'RLM', 'HRS', 'DCNAQ', 'AHM', 'FBHS', 'FTL.A', 'HMA', 'TIF', 'INCLF', 'GFS.A', 'KSU', 'ABC', 'AZA.A', 'AGN', 'CITGQ', 'CELG', 'FLT', 'FLIR', 'WLL', 'CEN', 'DALRQ', 'GRA', 'BFO', 'RTN', 'ATVI', 'ESV', 'CIT.A', 'MYL', 'SHN', 'XL', 'ONE', 'DNR', 'SFS', 'CBB', 'HFC', 'NCR', 'WPX', 'MDR', 'RHT', 'WCG', 'LLL', 'AGC', 'DISH', 'XLNX', 'CPNLQ', 'BHGE', 'MTL', 'TMK', 'VSTNQ', 'ARNC', 'NLSN', 'LSI', 'DISCA', 'FII', 'UTX', 'TSG', 'NBL', 'WIN', 'CERN', 'WFT', 'SYMC', 'VIAB', 'LOR', 'AKS', 'BT', 'DTV', 'GIDL', 'INFO', 'TEN', 'XEC', 'BBT', 'WLTW', 'ETFC', 'PKI', 'JP', 'ACKH', 'CDAY', 'BBI', 'TWTR', 'FTR', 'PEAK', 'QEP', 'TA', 'LDW.B', 'MXIM', 'BMGCA', 'FB', 'CTXS', 'ANTM', 'ABKFQ', 'CHA', 'CBS', 'JCP', 'ADS', 'NAV', 'SGID', 'PBCT', 'FRC', 'AVP', 'UMG', 'BLL', 'AABA', 'AT', 'CRR', 'NLOK', 'BVSN', 'CTB', 'NRTLQ', 'DISCK', 'VAR', 'CXO', 'I', 'RE', 'TSS', 'LM', 'CTL', 'APC', 'BGG', 'PVT', 'FLTWQ', 'MON', 'VIAC', 'JEC', 'BBBY', 

Adj Close                                                  \
                            A AABA AAL AAMRQ AAP      AAPL ABBV ABC ABI ABKFQ   
Date                                                                            
1996-01-02 00:00:00       NaN  NaN NaN   NaN NaN  0.242501  NaN NaN NaN   NaN   
1996-01-03 00:00:00       NaN  NaN NaN   NaN NaN  0.242501  NaN NaN NaN   NaN   
1996-01-04 00:00:00       NaN  NaN NaN   NaN NaN  0.238255  NaN NaN NaN   NaN   
1996-01-05 00:00:00       NaN  NaN NaN   NaN NaN  0.258543  NaN NaN NaN   NaN   
1996-01-08 00:00:00       NaN  NaN NaN   NaN NaN  0.261373  NaN NaN NaN   NaN   

                     ...   Volume                                               
                     ...      XRX XTO XYL YNR YRCW YUM ZBH    ZBRA    ZION ZTS  
Date                 ...                                                        
1996-01-02 00:00:00  ...   785565 NaN NaN NaN  NaN NaN NaN  471825  142400 NaN  
1996-01-03 00:00:00  ...  1064270 NaN NaN NaN  NaN NaN NaN  342900  352400 NaN  
1996-01-04 00:00:00  ...  1160815 NaN NaN NaN  NaN NaN NaN  472500  488000 NaN  
1996-01-05 00:00:00  ...  1064498 NaN NaN NaN  NaN NaN NaN  380475  120400 NaN  
1996-01-08 00:00:00  ...   258212 NaN NaN NaN  NaN NaN NaN   12375   98800 NaN  

[5 rows x 6978 columns]

In [13]:
adj_close = data["Adj Close"]
adj_close = adj_close.drop(columns=adj_close.columns[adj_close.count() < 1000])
adj_close.head()

,A,AAL,AAP,AAPL,ABBV,ABMD,ABT,ACGL,ACN,ACS,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1996-01-02 00:00:00,NaN,NaN,NaN,0.242501,NaN,7.250,4.985257,2.569444,NaN,NaN,...,7.238386,8.375014,5.400020,31.252378,NaN,NaN,NaN,15.388889,11.960916,NaN
1996-01-03 00:00:00,NaN,NaN,NaN,0.242501,NaN,7.375,5.000181,2.541667,NaN,NaN,...,7.329781,8.400988,5.468374,31.223751,NaN,NaN,NaN,15.666667,11.696796,NaN
1996-01-04 00:00:00,NaN,NaN,NaN,0.238255,NaN,7.500,5.074811,2.555556,NaN,NaN,...,7.238386,8.452925,5.451284,30.565504,NaN,NaN,NaN,14.000000,11.206285,NaN
1996-01-05 00:00:00,NaN,NaN,NaN,0.258543,NaN,7.500,4.970332,2.500000,NaN,NaN,...,7.293225,8.660674,5.365843,30.222082,NaN,NaN,NaN,13.444444,11.244011,NaN
1996-01-08 00:00:00,NaN,NaN,NaN,0.261373,NaN,7.125,5.059889,2.500000,NaN,NaN,...,7.256669,8.764545,5.331663,30.164845,NaN,NaN,NaN,13.444444,11.130821,NaN


In [26]:
adj_close.to_parquet("adj_close.parquet")

In [2]:
adj_close = pd.read_parquet("adj_close.parquet")

In [3]:
import numpy as np

def calc_log_rtn(close, n=1):
    # convert the close price to numpy array
    close = np.array(close, dtype=np.float64)
    
    # check n
    if n < 1:
        raise ValueError("n must be at least 1")
    
    # compute the log return, from n-th day to the end
    # assert that the close price is not 0 and not NaN
    divi_resu = np.full_like(close, np.nan)
    np.divide(close[n:], close[:-n], out=divi_resu[n:], where=(close[:-n] != 0))
    
    # compute the log return securely
    log_result = np.full_like(close, np.nan)
    where_condition = (divi_resu != 0) & (divi_resu != np.nan)
    np.log(divi_resu, out=log_result, where=where_condition)
    
    return log_result

def calc_lag_return(close: pd.Series, lag: int = 1) -> pd.Series:
    close = close.to_numpy(dtype=np.float64)
    result = np.full_like(close, -np.inf)
    result[:lag] = np.nan
    np.divide(close[lag:], close[:-lag], out=result[lag:], where=close[:-lag] != 0)
    return result - 1

return_tomorrow = adj_close.apply(calc_log_rtn).shift(-1).dropna(how="all", axis=0)
return_tomorrow.head()

,A,AAL,AAP,AAPL,ABBV,ABMD,ABT,ACGL,ACN,ACS,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1996-01-02,NaN,NaN,NaN,0.000000,NaN,0.017094,0.002989,-0.010869,NaN,NaN,...,0.012547,0.003096,0.012579,-0.000916,NaN,NaN,NaN,0.017890,-0.022329,NaN
1996-01-03,NaN,NaN,NaN,-0.017663,NaN,0.016807,0.014815,0.005450,NaN,NaN,...,-0.012547,0.006163,-0.003130,-0.021307,NaN,NaN,NaN,-0.112478,-0.042840,NaN
1996-01-04,NaN,NaN,NaN,0.081718,NaN,0.000000,-0.020803,-0.021979,NaN,NaN,...,0.007548,0.024280,-0.015798,-0.011299,NaN,NaN,NaN,-0.040491,0.003361,NaN
1996-01-05,NaN,NaN,NaN,0.010889,NaN,-0.051293,0.017858,0.000000,NaN,NaN,...,-0.005025,0.011922,-0.006390,-0.001896,NaN,NaN,NaN,0.000000,-0.010118,NaN
1996-01-08,NaN,NaN,NaN,-0.055673,NaN,0.000000,-0.007774,0.000000,NaN,NaN,...,0.002516,-0.019447,0.000000,-0.000949,NaN,NaN,NaN,0.000000,-0.013652,NaN


In [4]:
(return_tomorrow.count() - adj_close.count()).value_counts()

-1      692
-5        7
-4        7
-2        6
-6        4
       ... 
-27       1
-70       1
-43       1
-228      1
-115      1
Name: count, Length: 61, dtype: int64

In [5]:
return_tomorrow.max().sort_values()

FPC      0.027615
UST      0.043994
SUB      0.053966
MHS      0.066302
WMX      0.079964
          ...    
MEL      9.815383
TNB      9.903488
USBC    10.126651
CBE     10.434116
SBL     11.286938
Length: 791, dtype: float64

In [6]:
return_tomorrow.min().sort_values()

SBL   -11.247657
CBE   -10.434116
TNB    -9.916877
MEL    -9.809450
CNG    -9.294682
         ...    
GLD    -0.091905
WMX    -0.075523
SUB    -0.050500
UST    -0.036765
FPC    -0.027615
Length: 791, dtype: float64

In [7]:
s = return_tomorrow["CBE"]
s[s > 10]

Date
2015-11-05    10.392068
2015-11-11    10.392068
2015-11-13    10.392068
2015-11-18    10.392068
2015-11-23    10.392068
2015-12-04    10.434116
2015-12-09    10.434116
2015-12-14    10.434116
2015-12-16    10.434116
2015-12-21    10.434116
2015-12-30    10.434116
2016-01-04    10.434116
2016-01-06    10.434116
2016-01-11    10.434116
2016-01-14    10.434116
2016-01-21    10.434116
2016-01-25    10.434116
2016-02-01    10.434116
2016-02-08    10.434116
2016-02-10    10.434116
2016-02-16    10.434116
2016-02-22    10.434116
2016-02-26    10.434116
2017-07-26    10.306951
2017-08-29    10.240388
2017-09-15    10.312314
2017-09-21    10.312314
2017-09-25    10.312314
2017-09-27    10.312314
2017-10-04    10.312314
2017-11-29    10.325482
2017-12-28    10.308953
Name: CBE, dtype: float64

In [8]:
return_tomorrow.to_parquet("return_tomorrow.parquet")